In [ ]:
# INSTALLATION & IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════

!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install gradio pandas numpy scikit-learn

# Standard Library Imports
import os
import re
import json
import random
import pickle
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from collections import defaultdict, Counter

# Data Science & ML Imports
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

# Gradio for UI
import gradio as gr

# Suppress warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-j97x7gfw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-j97x7gfw
  Resolved https://github.com/huggingface/transformers.git to commit 6d0adb5bab99a1175bc77036a490d3779bb210ee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.9/520.9 kB 35.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=10870618 sha256=28a654643c465b32605713eb61f641667601a289174d27a1f67093edd51b85d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-9p2xub6e/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Fou

In [ ]:
# HUGGING FACE AUTHENTICATION & GOOGLE DRIVE SETUP
# ═══════════════════════════════════════════════════════════════════════════════

from huggingface_hub import login
from google.colab import drive

# Hugging Face Login
login()
hf_model = "epfl-llm/meditron-7b"

# Mount Google Drive
drive.mount('/content/drive')

# Define Google Drive Paths
DRIVE_BASE_PATH = "/content/drive/MyDrive/DrugBank/"
MODEL_SAVE_PATH = DRIVE_BASE_PATH + "complete_drug_model_system.pkl"
BACKUP_PATH = DRIVE_BASE_PATH + "drug_model.pkl"


Mounted at /content/drive


In [ ]:
# CONFIGURATION CLASSES
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class StreamlinedConfig:
    """Configuration for the drug recommendation system"""
    test_size: float = 0.2
    random_state: int = 42
    tfidf_max_features: int = 5000
    ngram_range: Tuple[int, int] = (1, 4)
    min_df: int = 1
    max_df: float = 0.9
    svd_components: int = 300
    fixed_k: int = 3
    ensemble_weights: List[float] = field(default_factory=lambda: ['0.337', '0.496', '0.167'])


In [ ]:
# DRUG RECOMMENDATION SYSTEM CLASS (ML MODEL)
# ═══════════════════════════════════════════════════════════════════════════════

class StreamlinedDrugSystem:
    """
    Main ML model for drug recommendations using TF-IDF + SVD ensemble
    Fixed K=3 recommendations
    """

    def __init__(self, config: StreamlinedConfig = None):
        self.config = config or StreamlinedConfig()
        self.df = None
        self.train_df = None
        self.test_df = None
        self.vectorizers = {}
        self.svd_models = {}
        self.embeddings_matrices = {}
        self.indication_to_drugs = None
        self.drug_to_indications = None
        self.drug_profiles = None
        self.optimal_k = self.config.fixed_k
        self.is_trained = False
        self.medical_synonyms = self._create_medical_synonyms()

    # ─────────────────────────────────────────────────────────────────────────
    # Medical Synonyms Dictionary
    # ─────────────────────────────────────────────────────────────────────────
    def _create_medical_synonyms(self):
        """Medical synonyms for text enhancement"""
        return {
            'pain': ['discomfort', 'ache', 'soreness', 'tenderness'],
            'infection': ['bacterial', 'pathogen', 'contamination'],
            'inflammation': ['swelling', 'irritation', 'inflammatory'],
            'treatment': ['therapy', 'medication', 'drug', 'medicine'],
            'disease': ['disorder', 'condition', 'illness', 'ailment'],
            'chronic': ['persistent', 'long-term', 'ongoing'],
            'acute': ['sudden', 'severe', 'immediate'],
            'cancer': ['tumor', 'malignancy', 'carcinoma', 'neoplasm'],
            'diabetes': ['diabetic', 'glucose', 'blood sugar'],
            'hypertension': ['high blood pressure', 'elevated pressure'],
            'depression': ['depressive', 'mood disorder', 'mental health'],
            'anxiety': ['anxious', 'panic', 'stress'],
            'allergy': ['allergic', 'hypersensitivity', 'reaction'],
            'asthma': ['respiratory', 'breathing', 'airways'],
            'heart': ['cardiac', 'cardiovascular', 'coronary'],
            'liver': ['hepatic', 'hepato'],
            'kidney': ['renal', 'nephro'],
            'brain': ['cerebral', 'neurological', 'neuro'],
            'bone': ['skeletal', 'osteo'],
            'muscle': ['muscular', 'myalgia'],
            'skin': ['dermal', 'cutaneous', 'dermatological'],
            'blood': ['hematological', 'hemic'],
            'immune': ['immunological', 'autoimmune'],
            'pediatric': ['children', 'child', 'infant', 'pediatric'],
            'adult': ['grown-up', 'mature'],
            'elderly': ['geriatric', 'senior', 'aged']
        }

    # ─────────────────────────────────────────────────────────────────────────
    # Text Processing Methods
    # ─────────────────────────────────────────────────────────────────────────
    def _enhance_text_with_synonyms(self, text: str) -> str:
        """Enhance text with medical synonyms"""
        enhanced_text = text
        words = text.lower().split()

        for word in words:
            if word in self.medical_synonyms:
                synonyms = self.medical_synonyms[word]
                selected_synonyms = random.sample(synonyms, min(2, len(synonyms)))
                enhanced_text += ' ' + ' '.join(selected_synonyms)

        return enhanced_text

    def _advanced_text_cleaning(self, text: str) -> str:
        """Advanced text cleaning with medical term preservation"""
        if pd.isna(text) or text == '':
            return ''

        text = str(text).lower()
        text = re.sub(r'\b(\w+)-(\w+)\b', r'\1 \2', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()

        words = text.split()
        filtered_words = []
        for word in words:
            if len(word) >= 2 or word.upper() in ['mg', 'ml', 'iv', 'po', 'pr']:
                filtered_words.append(word)

        return ' '.join(filtered_words)

    def _create_enhanced_indication(self, text: str) -> str:
        """Create enhanced indication text"""
        enhanced = text + ' ' + text
        enhanced += ' ' + self._enhance_text_with_synonyms(text)

        words = text.split()
        if len(words) > 1:
            important_words = [w for w in words if len(w) > 3]
            enhanced += ' ' + ' '.join(important_words)

        return enhanced.strip()

    def _normalize_drug_name(self, drug_name: str) -> str:
        """Normalize drug names"""
        if pd.isna(drug_name):
            return ''
        normalized = re.sub(r'[^a-zA-Z0-9]', '', str(drug_name).lower())
        return normalized

    # ─────────────────────────────────────────────────────────────────────────
    # Data Loading & Preparation
    # ─────────────────────────────────────────────────────────────────────────
    def load_and_prepare_data(self, filepath: str):
        """Load and prepare data from CSV"""
        self.df = pd.read_csv(filepath)

        # Handle column names
        if 'name' in self.df.columns and 'indication' in self.df.columns:
            self.df = self.df.rename(columns={'name': 'drug_name'})
        elif 'Name' in self.df.columns and 'Indication' in self.df.columns:
            self.df = self.df.rename(columns={'Name': 'drug_name', 'Indication': 'indication'})

        # Clean data
        self.df['drug_name'] = self.df['drug_name'].fillna('').astype(str)
        self.df['indication'] = self.df['indication'].fillna('').astype(str)
        self.df['clean_indication'] = self.df['indication'].apply(self._advanced_text_cleaning)
        self.df = self.df[self.df['clean_indication'].str.strip() != ''].reset_index(drop=True)
        self.df['normalized_drug_name'] = self.df['drug_name'].apply(self._normalize_drug_name)
        self.df['enhanced_indication'] = self.df['clean_indication'].apply(self._create_enhanced_indication)
        self.df['synonym_enhanced'] = self.df['clean_indication'].apply(self._enhance_text_with_synonyms)

        # Create mappings
        self.indication_to_drugs = defaultdict(set)
        self.drug_to_indications = defaultdict(set)
        self.drug_profiles = {}

        for _, row in self.df.iterrows():
            drug = row['normalized_drug_name']
            original_drug = row['drug_name']
            indication = row['clean_indication']

            self.drug_profiles[drug] = {
                'original_name': original_drug,
                'indication': indication,
                'enhanced_indication': row['enhanced_indication'],
                'synonym_enhanced': row['synonym_enhanced']
            }

            if indication:
                self.indication_to_drugs[indication].add(drug)
                self.drug_to_indications[drug].add(indication)

        # Split data
        self.train_df, self.test_df = train_test_split(
            self.df, test_size=self.config.test_size,
            random_state=self.config.random_state,
            shuffle=True
        )

    # ─────────────────────────────────────────────────────────────────────────
    # Model Training - Embedding Creation
    # ─────────────────────────────────────────────────────────────────────────
    def create_enhanced_embeddings(self):
        """Create TF-IDF embeddings with 3 strategies"""
        # Enhanced TF-IDF
        self.vectorizers['enhanced'] = TfidfVectorizer(
            max_features=self.config.tfidf_max_features,
            ngram_range=self.config.ngram_range,
            stop_words='english',
            min_df=self.config.min_df,
            max_df=self.config.max_df,
            sublinear_tf=True,
            norm='l2',
            analyzer='word'
        )

        # Character-level
        self.vectorizers['char'] = TfidfVectorizer(
            max_features=self.config.tfidf_max_features // 2,
            ngram_range=(3, 5),
            analyzer='char',
            stop_words='english',
            min_df=self.config.min_df,
            max_df=self.config.max_df,
            sublinear_tf=True,
            norm='l2'
        )

        # Word-level
        self.vectorizers['word'] = TfidfVectorizer(
            max_features=self.config.tfidf_max_features,
            ngram_range=(1, 2),
            stop_words='english',
            min_df=1,
            max_df=0.8,
            sublinear_tf=True,
            norm='l2'
        )

        # Text versions
        text_versions = {
            'enhanced': self.train_df['enhanced_indication'].tolist(),
            'char': self.train_df['clean_indication'].tolist(),
            'word': self.train_df['synonym_enhanced'].tolist()
        }

        # Create embeddings
        for strategy, vectorizer in self.vectorizers.items():
            tfidf_matrix = vectorizer.fit_transform(text_versions[strategy])

            self.svd_models[strategy] = TruncatedSVD(
                n_components=self.config.svd_components,
                random_state=self.config.random_state
            )

            embeddings = self.svd_models[strategy].fit_transform(tfidf_matrix)
            self.embeddings_matrices[strategy] = normalize(embeddings, norm='l2')

        return self.embeddings_matrices

    # ─────────────────────────────────────────────────────────────────────────
    # Prediction Methods
    # ─────────────────────────────────────────────────────────────────────────
    def _get_ensemble_query_embedding(self, query: Dict) -> Dict[str, np.ndarray]:
        """Get ensemble query embeddings"""
        indication_text = query.get('indication_text', '')

        enhanced_query = self._create_enhanced_indication(indication_text)
        synonym_query = self._enhance_text_with_synonyms(indication_text)

        query_embeddings = {}

        # Enhanced strategy
        query_tfidf = self.vectorizers['enhanced'].transform([enhanced_query])
        query_embeddings['enhanced'] = normalize(
            self.svd_models['enhanced'].transform(query_tfidf), norm='l2'
        )[0]

        # Character strategy
        query_tfidf = self.vectorizers['char'].transform([indication_text])
        query_embeddings['char'] = normalize(
            self.svd_models['char'].transform(query_tfidf), norm='l2'
        )[0]

        # Word strategy
        query_tfidf = self.vectorizers['word'].transform([synonym_query])
        query_embeddings['word'] = normalize(
            self.svd_models['word'].transform(query_tfidf), norm='l2'
        )[0]

        return query_embeddings

    def search_similar_drugs_ensemble(self, query_embeddings: Dict[str, np.ndarray], top_k: int = 3) -> Tuple[List[float], List[int]]:
        """Search using ensemble"""
        all_similarities = []

        for i, (strategy, weight) in enumerate(zip(['enhanced', 'char', 'word'], self.config.ensemble_weights)):
            query_embedding = query_embeddings[strategy].reshape(1, -1)
            similarities = cosine_similarity(query_embedding, self.embeddings_matrices[strategy])[0]
            all_similarities.append(similarities * weight)

        similarities = np.sum(all_similarities, axis=0)
        top_indices = np.argsort(similarities)[::-1][:top_k]
        top_scores = similarities[top_indices]

        return top_scores, top_indices

    def train_model(self):
        """Train the model with fixed K=3"""
        self.create_enhanced_embeddings()
        self.is_trained = True

    def predict_drugs_for_condition(self, condition: str, top_k: int = None) -> List[Dict]:
        """Main prediction method for Gradio interface"""
        if not self.is_trained:
            return []

        if top_k is None:
            top_k = self.optimal_k

        try:
            query = {'indication_text': condition}
            query_embeddings = self._get_ensemble_query_embedding(query)
            scores, indices = self.search_similar_drugs_ensemble(query_embeddings, top_k)

            recommendations = []
            for idx, score in zip(indices, scores):
                if idx < len(self.train_df):
                    drug_row = self.train_df.iloc[idx]
                    recommendations.append({
                        'drug_name': drug_row['drug_name'],
                        'normalized_name': drug_row['normalized_drug_name'],
                        'indication': drug_row['clean_indication'],
                        'confidence_score': float(score),
                        'rank': len(recommendations) + 1
                    })

            return recommendations

        except Exception as e:
            return []

    # ─────────────────────────────────────────────────────────────────────────
    # Model Saving
    # ─────────────────────────────────────────────────────────────────────────
    def save_complete_system_to_drive(self):
        """Save the complete system to Google Drive"""
        if not self.is_trained:
            return False

        try:
            os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

            # Save the entire system object
            joblib.dump(self, MODEL_SAVE_PATH, compress=3)

            # Backup with pickle
            with open(BACKUP_PATH, 'wb') as f:
                pickle.dump(self, f, protocol=pickle.HIGHEST_PROTOCOL)

            print(f"Model saved to: {MODEL_SAVE_PATH}")
            return True

        except Exception as e:
            return False


In [ ]:
# MODEL LOADING & TRAINING FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

def load_complete_drug_model(filepath=MODEL_SAVE_PATH):
    """Load the complete drug model for Gradio interface"""
    try:
        try:
            model = joblib.load(filepath)
        except Exception:
            backup_path = filepath.replace('.joblib', '.pkl')
            with open(backup_path, 'rb') as f:
                model = pickle.load(f)

        if hasattr(model, 'predict_drugs_for_condition'):
            return model
        else:
            return None

    except Exception:
        return None

def train_and_save_streamlined_system(data_filepath: str):
    """Train and save the streamlined system"""
    config = StreamlinedConfig(
        tfidf_max_features=5000,
        ngram_range=(1, 4),
        svd_components=300,
        fixed_k=3,
        ensemble_weights=[0.4, 0.3, 0.3]
    )

    system = StreamlinedDrugSystem(config)

    try:
        system.load_and_prepare_data(data_filepath)
        system.train_model()
        save_success = system.save_complete_system_to_drive()

        if save_success:
            return system
        else:
            return None

    except Exception:
        return None

In [ ]:
# MAIN EXECUTION - TRAIN MODEL
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    data_file = '/content/drive/MyDrive/DrugBank/DRUG_indications.csv'
    system = train_and_save_streamlined_system(data_file)


Model saved to: /content/drive/MyDrive/DrugBank/complete_drug_model_system.pkl


In [ ]:
# TRANSFORMERS LIBRARY SETUP & CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

warnings.filterwarnings('ignore')

# Try to import transformers (optional for adverse reaction model)
try:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    import torch
    TRANSFORMERS_AVAILABLE = True
    print("✅ Transformers library available")
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    print("⚠️ Transformers library not available")

# ═══════════════════════════════════════════════════════════════════════════════
# FILE PATH CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

DRUG_MODEL_PATH = "/content/drive/MyDrive/DrugBank/complete_drug_model_system.pkl"
PATIENT_DATA_PATH = "/content/drive/MyDrive/DrugBank/ER_data.csv"
DRUG_INTERACTIONS_PATH = "/content/drive/MyDrive/DrugBank/drug_interaction.csv"

print("✅ Configuration loaded successfully")
print(f"📊 Active paths:")
print(f"   Drug Model: {DRUG_MODEL_PATH}")
print(f"   Patient Data: {PATIENT_DATA_PATH}")
print(f"   Drug Interactions: {DRUG_INTERACTIONS_PATH}")
print(f"   Adverse Reactions: 🔄 MODEL LOADING ATTEMPTED")


✅ Transformers library available
✅ Configuration loaded successfully
📊 Active paths:
   Drug Model: /content/drive/MyDrive/DrugBank/complete_drug_model_system.pkl
   Patient Data: /content/drive/MyDrive/DrugBank/ER_data.csv
   Drug Interactions: /content/drive/MyDrive/DrugBank/drug_interaction.csv
   Adverse Reactions: 🔄 MODEL LOADING ATTEMPTED


In [ ]:
# DATA LOADING FUNCTIONS (Patient Data, Drug Model, Interactions)
# ═══════════════════════════════════════════════════════════════════════════════

def load_patient_data():
    """Load real patient data from CSV file"""
    try:
        if os.path.exists(PATIENT_DATA_PATH):
            df = pd.read_csv(PATIENT_DATA_PATH)
            required_columns = ['patient_id', 'name', 'age', 'gender', 'conditions', 'medications']
            missing_columns = [col for col in required_columns if col not in df.columns]

            if missing_columns:
                print(f"❌ Missing columns in patient data: {missing_columns}")
                return pd.DataFrame()

            df['name'] = df['name'].fillna('Unknown')
            df['conditions'] = df['conditions'].fillna('')
            df['medications'] = df['medications'].fillna('none')
            print(f"✅ Loaded {len(df)} patients from {PATIENT_DATA_PATH}")
            return df
        else:
            print(f"❌ Patient data file not found: {PATIENT_DATA_PATH}")
            return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error loading patient data: {e}")
        return pd.DataFrame()

def load_drug_prediction_model():
    """
    Load drug prediction model with robust error handling
    ✅ FIX 1: Auto-retrain if corrupted pickle detected
    """
    try:
        if os.path.exists(DRUG_MODEL_PATH):
            try:
                import joblib
                model = joblib.load(DRUG_MODEL_PATH)
                print(f"✅ Loaded drug prediction model from {DRUG_MODEL_PATH}")
                return model
            except Exception as joblib_error:
                print(f"⚠️ Joblib loading failed: {joblib_error}")
                print(f"   Attempting to retrain model...")

                # ✅ FIX 1: Auto-retrain if corrupted
                if os.path.exists('/content/drive/MyDrive/DrugBank/DRUG_indications.csv'):
                    try:
                        # Delete corrupted file
                        if os.path.exists(DRUG_MODEL_PATH):
                            os.remove(DRUG_MODEL_PATH)
                            print("   ✓ Deleted corrupted model file")

                        # Retrain
                        print("   🔄 Retraining model from scratch...")
                        retrained_model = train_and_save_streamlined_system('/content/drive/MyDrive/DrugBank/DRUG_indications.csv')
                        if retrained_model:
                            print("   ✅ Model retrained successfully!")
                            return retrained_model
                    except Exception as retrain_error:
                        print(f"   ❌ Retraining failed: {retrain_error}")

            try:
                with open(DRUG_MODEL_PATH, 'rb') as f:
                    model = pickle.load(f)
                print(f"✅ Loaded drug prediction model with pickle")
                return model
            except Exception as pickle_error:
                print(f"⚠️ Pickle loading failed: {pickle_error}")

        print(f"❌ Drug model file not found: {DRUG_MODEL_PATH}")
        return None
    except Exception as e:
        print(f"❌ Error loading drug model: {e}")
        return None

def load_drug_interactions_simple():
    """Simple function to check if CSV exists"""
    try:
        if os.path.exists(DRUG_INTERACTIONS_PATH):
            df = pd.read_csv(DRUG_INTERACTIONS_PATH)
            print(f"✅ Drug interactions CSV found: {len(df)} drugs")
            print(f"✅ Columns: {list(df.columns)}")
            return True
        else:
            print(f"❌ Drug interactions CSV not found: {DRUG_INTERACTIONS_PATH}")
            return False
    except Exception as e:
        print(f"❌ Error checking CSV: {e}")
        return False

In [ ]:
# ADVERSE REACTION MODEL (Transformer-based AI)
# ═══════════════════════════════════════════════════════════════════════════════

def load_adverse_reaction_model():
    """
    Load the adverse reaction model with proper error handling
    ✅ FIX 2: Add offload directory for large model
    """
    try:
        model_path = "/content/drive/MyDrive/meditron7b_faers_efficient/final_model"

        print("🔄 Loading adverse reaction model...")
        print(f"📂 Model path: {model_path}")

        # Check if model exists
        if not os.path.exists(model_path):
            print(f"❌ Model path not found: {model_path}")
            return None

        # Load model and tokenizer
        print("📥 Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        print("📥 Loading model with disk offloading...")

        # ✅ FIX 2: Add offload directory
        offload_dir = "/content/offload_dir"
        os.makedirs(offload_dir, exist_ok=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            offload_folder=offload_dir,      # ✅ ADDED
            offload_state_dict=True           # ✅ ADDED
        )

        # Setup device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️ Using device: {device}")

        # Set evaluation mode
        model.eval()

        print("✅ Adverse reaction model loaded successfully!")

        return {
            "model": model,
            "tokenizer": tokenizer,
            "device": device,
            "type": "huggingface_transformers"
        }

    except Exception as e:
        print(f"❌ Error loading adverse reaction model: {e}")
        return None

def predict_adverse_reactions_enhanced(recommended_drug, patient_age, patient_gender, current_medications, adverse_reaction_model=None):
    """Enhanced adverse reaction prediction for drug prescription scenarios"""

    if adverse_reaction_model is None:
        return "None"

    try:
        model = adverse_reaction_model["model"]
        tokenizer = adverse_reaction_model["tokenizer"]
        device = adverse_reaction_model["device"]

        # Create enhanced prompt for drug prescription scenario
        current_meds_text = current_medications if current_medications != 'none' else "No current medications"

        prompt = f"""### Instruction:
You are a clinical pharmacology expert. Given the patient information and current medications below, predict the possible adverse reactions and drug interactions that could occur when adding the specified new drug to their treatment regimen. Consider patient demographics, existing medications, and potential drug-drug interactions.

### Input:
Patient Demographics:
- Age: {patient_age} years old
- Gender: {patient_gender}

Current Medications: {current_meds_text}

New Drug Being Prescribed: {recommended_drug}

Task: Predict adverse reactions that could occur when adding {recommended_drug} to this patient's current medication regimen, considering their age, gender, and potential drug-drug interactions.

### Output:
"""

        print(f"🤖 Generating adverse reactions for {recommended_drug}...")

        # Tokenize the input
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generate model output
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=300,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode the generated output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the adverse reactions
        if "### Output:" in generated_text:
            reaction_text = generated_text.split("### Output:")[-1].strip()
        else:
            reaction_text = generated_text.replace(prompt, "").strip()

        # Clean up the output
        reaction_text = clean_adverse_reaction_output(reaction_text)

        print(f"✅ Generated adverse reaction prediction")

        return reaction_text

    except Exception as e:
        print(f"❌ Error generating adverse reactions: {e}")
        return "None"

def clean_adverse_reaction_output(text):
    """Clean and format the adverse reaction output"""
    # Remove any residual prompt text
    text = text.replace("### Instruction:", "").replace("### Input:", "").replace("### Output:", "")

    # Remove excessive whitespace
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    text = '\n'.join(lines)

    # Limit length if too long
    if len(text) > 500:
        text = text[:500] + "..."

    return text


In [ ]:
# SYSTEM INITIALIZATION - Load All Components
# ═══════════════════════════════════════════════════════════════════════════════

print("🔄 Loading system components...")

# Load all components
try:
    patients_df = load_patient_data()
    drug_model = load_drug_prediction_model()
    drug_interactions_available = load_drug_interactions_simple()

    # Load adverse reaction model
    print("🔄 Loading adverse reaction model...")
    adverse_reaction_model = load_adverse_reaction_model()
    if adverse_reaction_model:
        print("✅ Adverse reaction model loaded successfully!")
    else:
        print("⚠️ Adverse reaction model not available - will return 'None' for adverse reactions")

    print("✅ Core components loaded successfully!")

except Exception as e:
    print(f"❌ Error during component loading: {e}")
    # Don't raise - continue with available components
    adverse_reaction_model = None

# Global variables for state management
selected_patient_id = None
recommendations_history = []


🔄 Loading system components...
✅ Loaded 1306 patients from /content/drive/MyDrive/DrugBank/ER_data.csv
✅ Loaded drug prediction model from /content/drive/MyDrive/DrugBank/complete_drug_model_system.pkl
✅ Drug interactions CSV found: 4105 drugs
✅ Columns: ['name', 'drug-interactions']
🔄 Loading adverse reaction model...
🔄 Loading adverse reaction model...
📂 Model path: /content/drive/MyDrive/meditron7b_faers_efficient/final_model
📥 Loading tokenizer...
📥 Loading model with disk offloading...


config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

🖥️ Using device: cuda
✅ Adverse reaction model loaded successfully!
✅ Adverse reaction model loaded successfully!
✅ Core components loaded successfully!


In [ ]:
# CORE PREDICTION & SAFETY FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

def predict_multiple_drugs_for_condition(condition, patient_age=None, patient_gender=None, patient_medications=None, top_k=3):
    """Predict multiple drug options for condition using trained model"""
    try:
        if drug_model is None:
            print("❌ Drug prediction model not available")
            return ["Drug prediction model not available"]

        print(f"🔍 Predicting drugs for condition: {condition}")
        print(f"📊 Using model type: {type(drug_model)}")

        # Method 1: If it's the robust drug recommendation system with predict method
        if hasattr(drug_model, 'predict'):
            try:
                print("🤖 Using robust model predict method")
                result = drug_model.predict(condition, top_k=top_k)

                if isinstance(result, str) and result:
                    return [result]
                elif isinstance(result, list) and len(result) > 0:
                    drugs = []
                    for drug in result[:top_k]:
                        if isinstance(drug, dict):
                            drug_name = drug.get('drug_name', str(drug))
                        else:
                            drug_name = str(drug)
                        drugs.append(drug_name)
                    print(f"✅ Model returned {len(drugs)} drugs: {drugs}")
                    return drugs
                else:
                    print("⚠️ Model returned empty result")
                    return ["No recommendation available"]
            except Exception as e:
                print(f"⚠️ Robust model predict failed: {e}")

        # Method 2: If it has predict_drugs_for_condition method
        if hasattr(drug_model, 'predict_drugs_for_condition'):
            try:
                print("🤖 Using predict_drugs_for_condition method")
                result = drug_model.predict_drugs_for_condition(condition, top_k=top_k)

                if isinstance(result, list) and len(result) > 0:
                    drugs = []
                    for drug in result[:top_k]:
                        if isinstance(drug, dict):
                            drug_name = drug.get('drug_name', str(drug))
                        else:
                            drug_name = str(drug)
                        drugs.append(drug_name)
                    print(f"✅ Model returned {len(drugs)} drugs: {drugs}")
                    return drugs
                else:
                    print("⚠️ predict_drugs_for_condition returned empty result")
                    return ["No recommendation available"]
            except Exception as e:
                print(f"⚠️ predict_drugs_for_condition failed: {e}")

        # Method 3: If it's a LoadedDrugModel from robust loading
        if hasattr(drug_model, 'vectorizers') and hasattr(drug_model, 'train_df'):
            try:
                print("🤖 Using LoadedDrugModel with vectorizers")

                # Use the robust prediction logic
                from sklearn.feature_extraction.text import TfidfVectorizer
                from sklearn.metrics.pairwise import cosine_similarity
                from sklearn.preprocessing import normalize
                import numpy as np

                # Get query embeddings
                query_embeddings = {}
                strategies = ['enhanced', 'char', 'word']

                for strategy in strategies:
                    if strategy in drug_model.vectorizers and strategy in drug_model.svd_models:
                        try:
                            query_tfidf = drug_model.vectorizers[strategy].transform([condition])
                            query_emb = normalize(
                                drug_model.svd_models[strategy].transform(query_tfidf), norm='l2')[0]
                            query_embeddings[strategy] = query_emb
                        except Exception as strategy_error:
                            print(f"⚠️ Strategy {strategy} failed: {strategy_error}")

                if not query_embeddings:
                    print("❌ No valid embeddings generated")
                    return ["No recommendation available"]

                # Combine similarities
                weights = drug_model.config.get('ensemble_weights', ['0.337', '0.496', '0.167'])
                all_similarities = []

                for strategy, weight in zip(strategies, weights):
                    if strategy in query_embeddings and strategy in drug_model.embeddings_matrices:
                        query_emb = query_embeddings[strategy].reshape(1, -1)
                        sims = cosine_similarity(query_emb, drug_model.embeddings_matrices[strategy])[0]
                        all_similarities.append(sims * weight)

                if not all_similarities:
                    print("❌ No similarities computed")
                    return ["No recommendation available"]

                final_sims = np.sum(all_similarities, axis=0)
                top_indices = np.argsort(final_sims)[::-1][:top_k]

                # Get drug names from training data
                drugs = []
                if len(drug_model.train_df) > 0:
                    for idx in top_indices:
                        if idx < len(drug_model.train_df):
                            drug_row = drug_model.train_df.iloc[idx]
                            drug_name = drug_row.get('drug_name', f'Drug_{idx}')
                            drugs.append(drug_name)

                if drugs:
                    print(f"✅ Vectorizer model returned {len(drugs)} drugs: {drugs}")
                    return drugs
                else:
                    print("⚠️ No drugs found in training data")
                    return ["No recommendation available"]

            except Exception as e:
                print(f"⚠️ Vectorizer-based prediction failed: {e}")

        # Method 4: If it's a simple dictionary/lookup
        elif isinstance(drug_model, dict):
            print("🤖 Using dictionary lookup model")
            condition_clean = condition.lower().strip()
            result = drug_model.get(condition_clean, ['No recommendation available'])
            if isinstance(result, list):
                return result[:top_k]
            else:
                return [str(result)]

        # Method 5: If it's a scikit-learn model
        elif hasattr(drug_model, 'predict'):
            try:
                print("🤖 Using sklearn-style predict method")
                features = prepare_features_for_model(condition, patient_age, patient_gender, patient_medications)
                prediction = drug_model.predict([features])[0]
                return [str(prediction)]
            except Exception as e:
                print(f"⚠️ Sklearn prediction failed: {e}")

        # If no method works, return error
        print("❌ No compatible prediction method found")
        print(f"❌ Model type: {type(drug_model)}")
        print(f"❌ Available methods: {[method for method in dir(drug_model) if not method.startswith('_')]}")
        return ["No compatible prediction method found"]

    except Exception as e:
        print(f"❌ Error in drug prediction: {e}")
        return ["Error in prediction"]

def check_drug_safety(drug_name, current_medications):
    """Simple drug interaction checker - reads CSV directly"""
    if current_medications == 'none' or not current_medications:
        return True, [], "No current medications"

    current_meds = [med.strip().lower() for med in str(current_medications).split(',')]
    drug_lower = drug_name.lower().strip()

    print(f"\n🔍 CHECKING DRUG INTERACTIONS")
    print(f"📋 Prescribed: {drug_name}")
    print(f"💊 Current meds: {current_meds}")

    try:
        if os.path.exists(DRUG_INTERACTIONS_PATH):
            df = pd.read_csv(DRUG_INTERACTIONS_PATH)
            print(f"✅ CSV loaded: {len(df)} rows")

            detailed_interactions = []

            # Look for the prescribed drug in the CSV
            for _, row in df.iterrows():
                csv_drug_name = str(row['name']).lower().strip()

                if csv_drug_name == drug_lower:
                    print(f"✅ Found {drug_name} in database")

                    interactions_str = str(row['drug-interactions'])
                    if pd.notna(interactions_str) and interactions_str != 'nan':
                        interaction_ids = interactions_str.split()
                        print(f"🔗 Has {len(interaction_ids)} potential interactions")

                        # Check each patient medication
                        for current_med in current_meds:
                            current_med_clean = current_med.strip().lower()

                            # Simple matching
                            for interaction_id in interaction_ids:
                                interaction_name = interaction_id.lower().strip()

                                if (current_med_clean in interaction_name or
                                    interaction_name in current_med_clean or
                                    current_med_clean == interaction_name):

                                    print(f"⚠️ INTERACTION: {drug_name} ↔ {current_med_clean}")
                                    detailed_interactions.append({
                                        'interacting_drug': current_med_clean,
                                        'interaction_type': 'Direct match',
                                        'severity': 'Moderate',
                                        'description': f"{drug_name} interacts with {current_med_clean}"
                                    })
                                    break
                            else:
                                print(f"✅ Safe: {drug_name} + {current_med_clean}")
                    break
            else:
                print(f"ℹ️ {drug_name} not found in database")
        else:
            print(f"❌ CSV file not found")
            return True, [], "CSV file not found"

    except Exception as e:
        print(f"❌ Error: {e}")
        return True, [], f"Error: {e}"

    is_safe = len(detailed_interactions) == 0
    summary = f"Interactions: {', '.join([i['interacting_drug'] for i in detailed_interactions])}" if detailed_interactions else "No interactions"

    return is_safe, detailed_interactions, summary

def predict_adverse_reactions(recommended_drug, patient_age, patient_gender, current_medications):
    """Predict adverse reactions using the enhanced transformer model"""
    return predict_adverse_reactions_enhanced(
        recommended_drug,
        patient_age,
        patient_gender,
        current_medications,
        adverse_reaction_model
    )

def update_patient_medication(patient_id, new_medication, new_condition):
    """Update patient's medication and condition in the dataset"""
    global patients_df

    try:
        if patients_df.empty or patient_id not in patients_df['patient_id'].values:
            return False, "Patient not found"

        # Find patient row
        patient_idx = patients_df[patients_df['patient_id'] == patient_id].index[0]

        # Update medications
        current_meds = str(patients_df.loc[patient_idx, 'medications'])
        if current_meds == 'none' or current_meds == 'nan':
            updated_meds = new_medication
        else:
            current_meds_list = [med.strip() for med in current_meds.split(',')]
            if new_medication not in current_meds_list:
                current_meds_list.append(new_medication)
            updated_meds = ', '.join(current_meds_list)

        # Update conditions
        current_conditions = str(patients_df.loc[patient_idx, 'conditions'])
        if current_conditions == '' or current_conditions == 'nan':
            updated_conditions = new_condition
        else:
            current_conditions_list = [cond.strip() for cond in current_conditions.split(',')]
            if new_condition not in current_conditions_list:
                current_conditions_list.append(new_condition)
            updated_conditions = ', '.join(current_conditions_list)

        # Apply updates
        patients_df.loc[patient_idx, 'medications'] = updated_meds
        patients_df.loc[patient_idx, 'conditions'] = updated_conditions

        return True, f"✅ Added {new_medication} for {new_condition} to patient record"

    except Exception as e:
        return False, f"❌ Error updating patient: {e}"

In [ ]:
# PATIENT MANAGEMENT FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

def search_patients(search_term):
    """Search patients by ID, name, age, or gender"""
    if patients_df.empty:
        return pd.DataFrame()

    if not search_term:
        return patients_df

    search_term = str(search_term).lower()
    mask = (
        patients_df['patient_id'].str.lower().str.contains(search_term, na=False) |
        patients_df['name'].str.lower().str.contains(search_term, na=False) |
        patients_df['age'].astype(str).str.contains(search_term, na=False) |
        patients_df['gender'].str.lower().str.contains(search_term, na=False)
    )
    return patients_df[mask]

def select_patient(patient_id):
    """Select a patient and return their information"""
    global selected_patient_id
    selected_patient_id = patient_id

    if patient_id == "Select a patient..." or patient_id == "No patients available":
        return "No patient selected", "", "", "", "", pd.DataFrame()

    if patients_df.empty:
        return "No patient data loaded", "", "", "", "", pd.DataFrame()

    try:
        patient = patients_df[patients_df['patient_id'] == patient_id].iloc[0]
        info = f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 18px; color: white; margin: 15px 0; box-shadow: 0 10px 40px rgba(102,126,234,0.3); border: 1px solid rgba(255,255,255,0.1);">
            <div style="text-align: center; margin-bottom: 20px;">
                <h3 style="margin: 0; color: #f8fafc; font-weight: 700; font-size: 20px;">
                    👤 Selected Patient Profile
                </h3>
                <div style="background: rgba(255,255,255,0.3); height: 2px; width: 100px; margin: 10px auto; border-radius: 1px;"></div>
            </div>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin-bottom: 20px;">
                <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 12px; border-left: 3px solid #f1f5f9;">
                    <div style="color: #e0e7ff; font-weight: 500; font-size: 13px; margin-bottom: 5px;">Patient ID</div>
                    <div style="color: #f8fafc; font-weight: 700; font-size: 15px;">{patient_id[:16]}...</div>
                </div>
                <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 12px; border-left: 3px solid #f1f5f9;">
                    <div style="color: #e0e7ff; font-weight: 500; font-size: 13px; margin-bottom: 5px;">Full Name</div>
                    <div style="color: #f8fafc; font-weight: 700; font-size: 15px;">{patient['name']}</div>
                </div>
                <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 12px; border-left: 3px solid #f1f5f9;">
                    <div style="color: #e0e7ff; font-weight: 500; font-size: 13px; margin-bottom: 5px;">Age</div>
                    <div style="color: #f8fafc; font-weight: 700; font-size: 15px;">{patient['age']} years</div>
                </div>
                <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 12px; border-left: 3px solid #f1f5f9;">
                    <div style="color: #e0e7ff; font-weight: 500; font-size: 13px; margin-bottom: 5px;">Gender</div>
                    <div style="color: #f8fafc; font-weight: 700; font-size: 15px;">{patient['gender']}</div>
                </div>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 18px; border-radius: 12px;">
                <div style="margin-bottom: 12px;">
                    <div style="color: #e0e7ff; font-weight: 600; font-size: 14px; margin-bottom: 6px;">💊 Current Medications</div>
                    <div style="color: #f1f5f9; font-weight: 600; font-size: 15px; line-height: 1.4;">{patient['medications']}</div>
                </div>
                <div>
                    <div style="color: #e0e7ff; font-weight: 600; font-size: 14px; margin-bottom: 6px;">🏥 Medical Conditions</div>
                    <div style="color: #f1f5f9; font-weight: 600; font-size: 15px; line-height: 1.4;">{patient['conditions']}</div>
                </div>
            </div>
        </div>
        """

        return (
            info,
            str(patient['age']),
            patient['gender'],
            patient['medications'],
            patient['conditions'],
            patients_df
        )
    except Exception as e:
        return f"Error loading patient: {e}", "", "", "", "", pd.DataFrame()

def get_drug_recommendations(condition, patient_age, patient_gender, current_medications, patient_conditions):
    """Get multiple drug recommendations with detailed safety and adverse reaction analysis"""
    global recommendations_history

    if not condition or not condition.strip():
        return "❌ Please enter a condition to treat.", "", pd.DataFrame()

    if not selected_patient_id or selected_patient_id == "No patients available":
        return "❌ Please select a patient first.", "", pd.DataFrame()

    try:
        # Get multiple drug recommendations (k=3)
        recommended_drugs = predict_multiple_drugs_for_condition(condition, patient_age, patient_gender, current_medications, top_k=3)

        if not recommended_drugs or recommended_drugs == ["No recommendation available"]:
            return "❌ No drug recommendations available for this condition.", "", pd.DataFrame()

        # Create drug options with detailed safety and adverse reaction information
        drug_options = []
        for drug in recommended_drugs:
            if drug and drug != "No recommendation available":
                # Get detailed interaction information
                is_safe, detailed_interactions, interaction_summary = check_drug_safety(drug, current_medications)

                # Get adverse reaction prediction
                adverse_reactions = predict_adverse_reactions(drug, patient_age, patient_gender, current_medications)

                # Prepare interaction details for display
                interaction_details = ""
                if detailed_interactions:
                    interaction_details = " | ".join([
                        f"{interaction['interacting_drug']} ({interaction['severity']})"
                        for interaction in detailed_interactions
                    ])
                else:
                    interaction_details = "None detected"

                safety_status = "✅ SAFE" if is_safe else f"⚠️ INTERACTIONS"
                safety_color = "#22c55e" if is_safe else "#ef4444"

                drug_options.append({
                    'Drug Name': drug,
                    'Safety Status': safety_status,
                    'Interacting Drugs': interaction_details,
                    'Adverse Reactions': adverse_reactions[:100] + "..." if len(str(adverse_reactions)) > 100 else adverse_reactions,
                    'Color': safety_color,
                    'Full_Adverse_Reactions': adverse_reactions
                })

        # Create results display
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        patient_name = patients_df[patients_df['patient_id'] == selected_patient_id].iloc[0]['name']

        result_html = f"""
        <div style="background: linear-gradient(135deg, #1e293b 0%, #334155 100%); padding: 30px; border-radius: 20px; color: white; margin: 15px 0; box-shadow: 0 10px 40px rgba(0,0,0,0.3); border: 1px solid #475569;">
            <div style="text-align: center; margin-bottom: 25px;">
                <h2 style="color: #f1f5f9; margin: 0 0 10px 0; font-weight: 700; font-size: 24px;">
                    💊 AI Drug Recommendations
                </h2>
                <div style="background: linear-gradient(90deg, #3b82f6, #8b5cf6); height: 3px; width: 100px; margin: 0 auto; border-radius: 2px;"></div>
            </div>

            <div style="background: rgba(248,250,252,0.1); padding: 20px; border-radius: 15px; margin-bottom: 20px; border: 1px solid rgba(248,250,252,0.1);">
                <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; font-size: 15px;">
                    <div style="background: rgba(59,130,246,0.2); padding: 15px; border-radius: 10px; border-left: 4px solid #3b82f6;">
                        <div style="color: #cbd5e1; font-weight: 500; margin-bottom: 5px;">Medical Condition</div>
                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px;">{condition}</div>
                    </div>
                    <div style="background: rgba(139,92,246,0.2); padding: 15px; border-radius: 10px; border-left: 4px solid #8b5cf6;">
                        <div style="color: #cbd5e1; font-weight: 500; margin-bottom: 5px;">Patient Profile</div>
                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px;">{patient_name}</div>
                    </div>
                    <div style="background: rgba(16,185,129,0.2); padding: 15px; border-radius: 10px; border-left: 4px solid #10b981;">
                        <div style="color: #cbd5e1; font-weight: 500; margin-bottom: 5px;">Analysis Time</div>
                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px;">{timestamp}</div>
                    </div>
                    <div style="background: rgba(245,158,11,0.2); padding: 15px; border-radius: 10px; border-left: 4px solid #f59e0b;">
                        <div style="color: #cbd5e1; font-weight: 500; margin-bottom: 5px;">Current Medications</div>
                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px;">{current_medications}</div>
                    </div>
                </div>
            </div>

            <div style="text-align: center; background: rgba(34,197,94,0.1); padding: 15px; border-radius: 12px; border: 1px solid rgba(34,197,94,0.3);">
                <div style="color: #bbf7d0; font-weight: 600; font-size: 15px; margin-bottom: 5px;">
                    🎯 {len(drug_options)} Personalized Recommendations Generated
                </div>
                <div style="color: #d1fae5; font-size: 13px; font-weight: 500;">
                    Click any drug name below to add it to the patient's treatment plan
                </div>
            </div>
        </div>
        """

        # Create drug selection DataFrame
        df_display = pd.DataFrame([
            {
                'Drug Name': option['Drug Name'],
                'Safety Status': option['Safety Status'],
                'Interacting Drugs': option['Interacting Drugs'],
                'Predicted Adverse Reactions': option['Adverse Reactions']
            } for option in drug_options
        ])

        return result_html, f"Click on any drug name below to prescribe it to the patient:", df_display

    except Exception as e:
        return f"❌ Error generating recommendations: {e}", "", pd.DataFrame()

def add_drug_to_patient(drug_name, condition, evt: gr.SelectData):
    """Add selected drug to patient record"""
    if not selected_patient_id:
        return "❌ No patient selected", pd.DataFrame()

    if not drug_name:
        return "❌ No drug selected", pd.DataFrame()

    # Update patient record
    success, message = update_patient_medication(selected_patient_id, drug_name, condition)

    if success:
        # Update recommendations history
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        patient_name = patients_df[patients_df['patient_id'] == selected_patient_id].iloc[0]['name']

        recommendations_history.append({
            'timestamp': timestamp,
            'patient_id': selected_patient_id,
            'patient_name': patient_name,
            'condition': condition,
            'prescribed_drug': drug_name
        })

        return message, patients_df
    else:
        return message, patients_df

In [ ]:
# GRADIO UI - CUSTOM CSS STYLING
# ═══════════════════════════════════════════════════════════════════════════════

custom_css = """
/* Dark Professional Theme */
.gradio-container {
    background: linear-gradient(135deg, #0f172a 0%, #1e293b 100%);
    font-family: 'Inter', 'Segoe UI', sans-serif;
    color: #e2e8f0;
}

.dark {
    --background: #0f172a;
    --foreground: #e2e8f0;
    --card: #1e293b;
    --card-foreground: #e2e8f0;
    --popover: #1e293b;
    --popover-foreground: #e2e8f0;
    --primary: #3b82f6;
    --primary-foreground: #f8fafc;
    --secondary: #475569;
    --secondary-foreground: #f1f5f9;
    --muted: #334155;
    --muted-foreground: #94a3b8;
    --accent: #475569;
    --accent-foreground: #f1f5f9;
    --destructive: #ef4444;
    --destructive-foreground: #fef2f2;
    --border: #334155;
    --input: #334155;
    --ring: #3b82f6;
}

/* Tab styling */
.tab-nav {
    background: #1e293b !important;
    border-bottom: 2px solid #334155;
}

.tab-nav button {
    background: transparent !important;
    color: #94a3b8 !important;
    border: none !important;
    padding: 12px 24px !important;
    margin: 0 4px !important;
    border-radius: 8px 8px 0 0 !important;
    transition: all 0.3s ease !important;
}

.tab-nav button.selected {
    background: linear-gradient(135deg, #3b82f6, #1d4ed8) !important;
    color: white !important;
    box-shadow: 0 4px 12px rgba(59, 130, 246, 0.3) !important;
}

.tab-nav button:hover {
    background: #334155 !important;
    color: #e2e8f0 !important;
}

/* Card styling */
.card {
    background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
    border: 1px solid #475569;
    border-radius: 12px;
    padding: 20px;
    margin: 10px 0;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
}

/* Input styling */
.gr-textbox, .gr-dropdown {
    background: #334155 !important;
    border: 2px solid #475569 !important;
    color: #e2e8f0 !important;
    border-radius: 8px !important;
}

.gr-textbox:focus, .gr-dropdown:focus {
    border-color: #3b82f6 !important;
    box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.1) !important;
}

/* Button styling */
.gr-button {
    background: linear-gradient(135deg, #3b82f6, #1d4ed8) !important;
    border: none !important;
    color: white !important;
    border-radius: 8px !important;
    padding: 12px 24px !important;
    font-weight: 600 !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 12px rgba(59, 130, 246, 0.3) !important;
}

.gr-button:hover {
    background: linear-gradient(135deg, #2563eb, #1d4ed8) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 20px rgba(59, 130, 246, 0.4) !important;
}

/* DataTable styling */
.gr-dataframe {
    background: #1e293b !important;
    border: 1px solid #475569 !important;
    border-radius: 8px !important;
}

.gr-dataframe table {
    background: #1e293b !important;
    color: #e2e8f0 !important;
}

.gr-dataframe th {
    background: #334155 !important;
    color: #f1f5f9 !important;
    font-weight: 600 !important;
    padding: 12px !important;
}

.gr-dataframe td {
    padding: 10px !important;
    border-bottom: 1px solid #475569 !important;
}

.gr-dataframe tr:hover {
    background: #334155 !important;
}

/* Drug name styling */
.drug-name {
    font-size: 18px !important;
    font-weight: bold !important;
    color: #60a5fa !important;
    cursor: pointer !important;
    padding: 8px 12px !important;
    border-radius: 6px !important;
    transition: all 0.3s ease !important;
    text-decoration: underline !important;
}

.drug-name:hover {
    background: #334155 !important;
    transform: scale(1.05) !important;
}

.safe-drug {
    color: #22c55e !important;
    border-left: 4px solid #22c55e !important;
}

.unsafe-drug {
    color: #ef4444 !important;
    border-left: 4px solid #ef4444 !important;
}

/* Header styling */
.header {
    background: linear-gradient(135deg, #1e40af 0%, #7c3aed 100%);
    padding: 30px;
    border-radius: 15px;
    text-align: center;
    margin-bottom: 30px;
    box-shadow: 0 10px 40px rgba(0, 0, 0, 0.3);
}

.header h1 {
    color: white;
    font-size: 2.5em;
    margin: 0 0 10px 0;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
}

.header p {
    color: #e0e7ff;
    font-size: 1.1em;
    margin: 5px 0;
}

/* Status indicators */
.status-indicator {
    display: inline-block;
    padding: 4px 8px;
    border-radius: 4px;
    font-size: 12px;
    font-weight: bold;
    margin: 2px;
}

.status-safe {
    background: #22c55e;
    color: white;
}

.status-warning {
    background: #ef4444;
    color: white;
}
"""

In [ ]:
# GRADIO INTERFACE - TAB 1: PATIENT DATABASE
# ═══════════════════════════════════════════════════════════════════════════════

# Create Gradio interface
with gr.Blocks(css=custom_css, title="Professional Drug Recommendation System", theme=gr.themes.Soft()) as demo:

    # HEADER
    gr.HTML("""
    <div style="background: linear-gradient(135deg, #1e40af 0%, #7c3aed 50%, #db2777 100%); padding: 35px; border-radius: 20px; text-align: center; margin-bottom: 30px; box-shadow: 0 15px 50px rgba(0, 0, 0, 0.4); position: relative; overflow: hidden;">
        <div style="position: absolute; top: 0; left: 0; right: 0; bottom: 0; background: rgba(255,255,255,0.05); backdrop-filter: blur(10px);"></div>
        <div style="position: relative; z-index: 2;">
            <h1 style="color: #f8fafc; font-size: 2.8em; margin: 0 0 15px 0; text-shadow: 2px 2px 8px rgba(0, 0, 0, 0.5); font-weight: 800; letter-spacing: -1px;">
                💊 Professional Drug Recommendation System
            </h1>
            <div style="background: linear-gradient(90deg, #f8fafc, transparent, #f8fafc); height: 2px; width: 80%; margin: 0 auto 20px auto; border-radius: 1px;"></div>
            <p style="color: #e0e7ff; font-size: 1.3em; margin: 10px 0; font-weight: 600; text-shadow: 1px 1px 4px rgba(0, 0, 0, 0.3);">
                AI-Powered Medical Decision Support for Healthcare Professionals
            </p>
            <div style="display: flex; justify-content: center; gap: 30px; margin-top: 20px; flex-wrap: wrap;">
                <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; color: #f1f5f9; font-size: 0.9em; font-weight: 600; backdrop-filter: blur(10px);">
                    🧠 Advanced AI Prediction
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; color: #f1f5f9; font-size: 0.9em; font-weight: 600; backdrop-filter: blur(10px);">
                    🔍 Real-time Safety Analysis
                </span>
                <span style="background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 25px; color: #f1f5f9; font-size: 0.9em; font-weight: 600; backdrop-filter: blur(10px);">
                    📊 Patient Data Integration
                </span>
            </div>
        </div>
    </div>
    """)

    with gr.Tabs():

        # ─────────────────────────────────────────────────────────────────────
        # TAB 1: PATIENT DATABASE
        # ─────────────────────────────────────────────────────────────────────
        with gr.TabItem("👥 Patient Database"):
            gr.HTML("<h2 style='color: #60a5fa; margin-bottom: 20px;'>🔍 Patient Search & Selection</h2>")

            with gr.Row():
                with gr.Column(scale=2):
                    search_input = gr.Textbox(
                        label="🔍 Search Patients",
                        placeholder="Enter patient ID, name, age, or gender...",
                        container=False
                    )
                with gr.Column(scale=1):
                    refresh_btn = gr.Button("🔄 Refresh", variant="secondary")

            patient_dropdown = gr.Dropdown(
                choices=["No patients available"] if patients_df.empty else ["Select a patient..."] + patients_df['patient_id'].tolist(),
                label="👤 Select Patient",
                value="No patients available" if patients_df.empty else "Select a patient...",
                container=False
            )

            # Patient information display
            patient_info_display = gr.HTML(
                value="""
                <div style="background: linear-gradient(135deg, #334155 0%, #475569 100%); padding: 30px; border-radius: 15px; text-align: center; color: #e2e8f0; box-shadow: 0 6px 24px rgba(0,0,0,0.2); border: 1px solid #64748b;">
                    <div style="margin-bottom: 15px;">
                        <span style="font-size: 48px; opacity: 0.7;">👥</span>
                    </div>
                    <h3 style="color: #f1f5f9; margin: 0 0 10px 0; font-weight: 700; font-size: 18px;">
                        Select a Patient Profile
                    </h3>
                    <p style="color: #cbd5e1; margin: 0; font-size: 14px; font-weight: 500;">
                        Choose a patient from the dropdown or search above to view their complete medical profile
                    </p>
                </div>
                """
            )

            # Display filtered dataset
            dataset_display = gr.Dataframe(
                value=patients_df,
                label="📋 Patient Database",
                interactive=False,
                wrap=True
            )

            # Hidden components to store patient data
            patient_age = gr.Textbox(visible=False)
            patient_gender = gr.Textbox(visible=False)
            patient_medications = gr.Textbox(visible=False)
            patient_conditions = gr.Textbox(visible=False)


            # GRADIO INTERFACE - TAB 2: DRUG PRESCRIPTION
# ═══════════════════════════════════════════════════════════════════════════════

        # ─────────────────────────────────────────────────────────────────────
        # TAB 2: DRUG RECOMMENDATIONS
        # ─────────────────────────────────────────────────────────────────────
        with gr.TabItem("💊 Drug Prescription"):
            gr.HTML("<h2 style='color: #60a5fa; margin-bottom: 20px;'>🎯 AI-Powered Drug Recommendations</h2>")

            with gr.Row():
                with gr.Column(scale=2):
                    condition_input = gr.Textbox(
                        label="🏥 Condition to Treat",
                        placeholder="e.g., diabetes, hypertension, headache, fever, infection...",
                        lines=2,
                        container=False
                    )

                    recommend_btn = gr.Button(
                        "🔍 Get Drug Recommendations",
                        variant="primary",
                        size="lg"
                    )

                with gr.Column(scale=1):
                    gr.HTML("""
                    <div style="background: linear-gradient(135deg, #1e40af 0%, #3730a3 100%); padding: 25px; border-radius: 15px; box-shadow: 0 8px 32px rgba(30, 64, 175, 0.3);">
                        <h3 style="color: #e0e7ff; margin: 0 0 20px 0; font-weight: 700; text-align: center; font-size: 18px;">
                            ⚡ Smart Prescription Workflow
                        </h3>
                        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 12px; border: 1px solid rgba(224,231,255,0.2);">
                            <div style="color: #e0e7ff; line-height: 2; font-size: 15px; font-weight: 600;">
                                <div style="margin: 8px 0; padding: 8px 0; border-left: 3px solid #60a5fa; padding-left: 15px;">
                                    <span style="color: #fbbf24;">1.</span> Select Patient Profile
                                </div>
                                <div style="margin: 8px 0; padding: 8px 0; border-left: 3px solid #60a5fa; padding-left: 15px;">
                                    <span style="color: #fbbf24;">2.</span> Enter Medical Condition
                                </div>
                                <div style="margin: 8px 0; padding: 8px 0; border-left: 3px solid #60a5fa; padding-left: 15px;">
                                    <span style="color: #fbbf24;">3.</span> AI Analyzes & Recommends
                                </div>
                                <div style="margin: 8px 0; padding: 8px 0; border-left: 3px solid #60a5fa; padding-left: 15px;">
                                    <span style="color: #fbbf24;">4.</span> Safety Check & Interactions
                                </div>
                                <div style="margin: 8px 0; padding: 8px 0; border-left: 3px solid #60a5fa; padding-left: 15px;">
                                    <span style="color: #fbbf24;">5.</span> Prescribe with Confidence
                                </div>
                            </div>
                        </div>
                        <div style="text-align: center; margin-top: 15px;">
                            <span style="color: #cbd5e1; font-size: 13px; font-weight: 500;">
                                🔬 Powered by Advanced AI Models
                            </span>
                        </div>
                    </div>
                    """)

            # Results display
            recommendation_output = gr.HTML(
                value="""
                <div style="background: linear-gradient(135deg, #334155 0%, #475569 100%); padding: 35px; border-radius: 15px; text-align: center; color: #e2e8f0; box-shadow: 0 6px 24px rgba(0,0,0,0.2); border: 1px solid #64748b;">
                    <div style="margin-bottom: 20px;">
                        <span style="font-size: 64px; opacity: 0.6;">🎯</span>
                    </div>
                    <h3 style="color: #f1f5f9; margin: 0 0 15px 0; font-weight: 700; font-size: 20px;">
                        Ready for AI-Powered Recommendations
                    </h3>
                    <p style="color: #cbd5e1; margin: 0 0 10px 0; font-size: 15px; font-weight: 500;">
                        Select a patient profile and enter a medical condition to receive intelligent drug recommendations
                    </p>
                    <div style="background: rgba(59,130,246,0.1); padding: 12px; border-radius: 8px; margin-top: 15px; border: 1px solid rgba(59,130,246,0.3);">
                        <span style="color: #bfdbfe; font-size: 13px; font-weight: 600;">
                            🤖 Advanced AI • 🔍 Safety Analysis • ⚡ Real-time Processing
                        </span>
                    </div>
                </div>
                """
            )

            selection_instruction = gr.HTML(visible=False)

            # Drug options table (clickable)
            drug_options_table = gr.Dataframe(
                label="💊 Available Drug Options - Click drug name to prescribe | Green=Safe | Red=Has interactions",
                interactive=False,
                wrap=True,
                visible=False
            )

            # Detailed adverse reaction display
            adverse_detail_display = gr.HTML(
                value="",
                label="🔬 Detailed Adverse Reaction Analysis",
                visible=False
            )

            # Status message
            status_message = gr.HTML(value="")

            # GRADIO INTERFACE - TAB 3: HISTORY & MODELS
# ═══════════════════════════════════════════════════════════════════════════════

        # ─────────────────────────────────────────────────────────────────────
        # TAB 3: PRESCRIPTION HISTORY & MODEL STATUS
        # ─────────────────────────────────────────────────────────────────────
        with gr.TabItem("📚 History & Models"):
            gr.HTML("<h2 style='color: #60a5fa; margin-bottom: 20px;'>📋 System Status & History</h2>")

            with gr.Row():
                with gr.Column():
                    # Model status display
                    gr.HTML(f"""
                    <div style="background: linear-gradient(135deg, #1e293b 0%, #334155 100%); padding: 25px; border-radius: 15px; margin-bottom: 25px; box-shadow: 0 8px 32px rgba(0,0,0,0.3); border: 1px solid #475569;">
                        <div style="text-align: center; margin-bottom: 20px;">
                            <h3 style="color: #f1f5f9; margin: 0; font-weight: 700; font-size: 20px;">
                                🤖 AI System Status
                            </h3>
                            <div style="background: linear-gradient(90deg, #f59e0b, #ef4444); height: 2px; width: 80px; margin: 10px auto; border-radius: 1px;"></div>
                        </div>
                        <div style="display: grid; gap: 15px;">
                            <div style="background: {'rgba(34,197,94,0.2)' if drug_model else 'rgba(239,68,68,0.2)'}; padding: 18px; border-radius: 12px; border-left: 4px solid {'#22c55e' if drug_model else '#ef4444'};">
                                <div style="display: flex; align-items: center; gap: 12px;">
                                    <span style="font-size: 24px;">🧠</span>
                                    <div>
                                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px; margin-bottom: 4px;">Drug Prediction Engine</div>
                                        <div style="color: {'#bbf7d0' if drug_model else '#fecaca'}; font-weight: 600; font-size: 14px;">
                                            {'✅ Active & Ready (k=3 recommendations)' if drug_model else '❌ Not Loaded'}
                                        </div>
                                    </div>
                                </div>
                            </div>

                            <div style="background: {'rgba(34,197,94,0.2)' if drug_interactions_available else 'rgba(239,68,68,0.2)'}; padding: 18px; border-radius: 12px; border-left: 4px solid {'#22c55e' if drug_interactions_available else '#ef4444'};">
                                <div style="display: flex; align-items: center; gap: 12px;">
                                    <span style="font-size: 24px;">⚠️</span>
                                    <div>
                                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px; margin-bottom: 4px;">Drug Interaction Checker</div>
                                        <div style="color: {'#bbf7d0' if drug_interactions_available else '#fecaca'}; font-weight: 600; font-size: 14px;">
                                            {'✅ Database Connected & Monitoring' if drug_interactions_available else '❌ Database Not Found'}
                                        </div>
                                    </div>
                                </div>
                            </div>

                            <div style="background: {'rgba(34,197,94,0.2)' if adverse_reaction_model else 'rgba(245,158,11,0.2)'}; padding: 18px; border-radius: 12px; border-left: 4px solid {'#22c55e' if adverse_reaction_model else '#f59e0b'};">
                                <div style="display: flex; align-items: center; gap: 12px;">
                                    <span style="font-size: 24px;">🧬</span>
                                    <div>
                                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px; margin-bottom: 4px;">Adverse Reaction Predictor</div>
                                        <div style="color: {'#bbf7d0' if adverse_reaction_model else '#fde68a'}; font-weight: 600; font-size: 14px;">
                                            {'✅ AI Model Active & Learning' if adverse_reaction_model else '⚠️ Returns "None" when unavailable'}
                                        </div>
                                    </div>
                                </div>
                            </div>

                            <div style="background: {'rgba(34,197,94,0.2)' if not patients_df.empty else 'rgba(239,68,68,0.2)'}; padding: 18px; border-radius: 12px; border-left: 4px solid {'#22c55e' if not patients_df.empty else '#ef4444'};">
                                <div style="display: flex; align-items: center; gap: 12px;">
                                    <span style="font-size: 24px;">👥</span>
                                    <div>
                                        <div style="color: #f1f5f9; font-weight: 700; font-size: 16px; margin-bottom: 4px;">Patient Database</div>
                                        <div style="color: {'#bbf7d0' if not patients_df.empty else '#fecaca'}; font-weight: 600; font-size: 14px;">
                                            {'✅ ' + str(len(patients_df)) + ' Patients Loaded & Ready' if not patients_df.empty else '❌ No Patient Data Available'}
                                        </div>
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>
                    """)

                    # Adverse reaction model status
                    if adverse_reaction_model:
                        gr.HTML("""
                        <div style="background: linear-gradient(135deg, #065f46 0%, #047857 100%); padding: 20px; border-radius: 15px; margin-bottom: 20px; box-shadow: 0 6px 24px rgba(5,95,70,0.3); border: 1px solid #059669;">
                            <div style="display: flex; align-items: center; gap: 15px;">
                                <div style="background: rgba(255,255,255,0.2); padding: 12px; border-radius: 50%; display: flex; align-items: center; justify-content: center;">
                                    <span style="font-size: 24px;">🚀</span>
                                </div>
                                <div>
                                    <h4 style="color: #f0fdf4; margin: 0 0 8px 0; font-weight: 700; font-size: 18px;">
                                        AI Model Fully Operational
                                    </h4>
                                    <p style="color: #bbf7d0; margin: 0; font-size: 14px; font-weight: 500; line-height: 1.4;">
                                        Advanced transformer model is loaded and generating intelligent adverse reaction predictions
                                    </p>
                                </div>
                            </div>
                        </div>
                        """)
                    else:
                        gr.HTML("""
                        <div style="background: linear-gradient(135deg, #475569 0%, #64748b 100%); padding: 20px; border-radius: 15px; margin-bottom: 20px; box-shadow: 0 6px 24px rgba(71,85,105,0.3); border: 1px solid #94a3b8;">
                            <div style="display: flex; align-items: center; gap: 15px;">
                                <div style="background: rgba(255,255,255,0.2); padding: 12px; border-radius: 50%; display: flex; align-items: center; justify-content: center;">
                                    <span style="font-size: 24px;">📄</span>
                                </div>
                                <div>
                                    <h4 style="color: #f1f5f9; margin: 0 0 8px 0; font-weight: 700; font-size: 18px;">
                                        Clean Mode Active
                                    </h4>
                                    <p style="color: #cbd5e1; margin: 0; font-size: 14px; font-weight: 500; line-height: 1.4;">
                                        Adverse reactions show "None" when AI model is unavailable - no placeholder text
                                    </p>
                                </div>
                            </div>
                        </div>
                        """)

                with gr.Column():
                    # History display
                    history_display = gr.HTML(
                        value="""
                        <div style="background: linear-gradient(135deg, #334155 0%, #475569 100%); padding: 30px; border-radius: 15px; text-align: center; color: #e2e8f0; box-shadow: 0 6px 24px rgba(0,0,0,0.2); border: 1px solid #64748b;">
                            <div style="margin-bottom: 15px;">
                                <span style="font-size: 48px; opacity: 0.7;">📚</span>
                            </div>
                            <h3 style="color: #f1f5f9; margin: 0 0 10px 0; font-weight: 700; font-size: 18px;">
                                Prescription History
                            </h3>
                            <p style="color: #cbd5e1; margin: 0; font-size: 14px; font-weight: 500; line-height: 1.5;">
                                Your prescription history and AI recommendations will appear here after you start prescribing medications to patients
                            </p>
                            <div style="background: rgba(34,197,94,0.1); padding: 10px; border-radius: 8px; margin-top: 15px; border: 1px solid rgba(34,197,94,0.3);">
                                <span style="color: #bbf7d0; font-size: 12px; font-weight: 600;">
                                    Ready to track all clinical decisions and AI insights
                                </span>
                            </div>
                        </div>
                        """
                    )

    # Footer
    gr.HTML(f"""
    <div style="background: #1e293b; padding: 20px; border-radius: 10px; margin-top: 30px; text-align: center; border: 1px solid #334155;">
        <p style="color: #94a3b8; margin: 0; font-size: 0.9em;">
            <strong>⚕️ Medical Disclaimer:</strong> This system is for healthcare professional use only.
            All recommendations must be validated by qualified medical practitioners.
        </p>
        <p style="color: #64748b; margin: 5px 0 0 0; font-size: 0.8em;">
            🔒 Ensure patient data compliance with healthcare regulations
        </p>
        <p style="color: #60a5fa; margin: 5px 0 0 0; font-size: 0.8em;">
            {'✅ <strong>AI MODEL ACTIVE:</strong> Advanced adverse reaction predictions enabled' if adverse_reaction_model else '📄 <strong>CLEAN MODE:</strong> Adverse reactions return "None" when AI unavailable'}
        </p>
    </div>
    """)

    # EVENT HANDLERS - Connect UI to Functions
# ═══════════════════════════════════════════════════════════════════════════════

    # Search functionality
    search_input.change(
        fn=lambda search_term: search_patients(search_term),
        inputs=[search_input],
        outputs=[dataset_display]
    )

    # Patient selection
    patient_dropdown.change(
        fn=select_patient,
        inputs=[patient_dropdown],
        outputs=[patient_info_display, patient_age, patient_gender, patient_medications, patient_conditions, dataset_display]
    )

    # Refresh patient list
    def refresh_patient_list():
        if patients_df.empty:
            return gr.update(choices=["No patients available"], value="No patients available")
        else:
            return gr.update(choices=["Select a patient..."] + patients_df['patient_id'].tolist(), value="Select a patient...")

    refresh_btn.click(
        fn=refresh_patient_list,
        outputs=[patient_dropdown]
    )

    # Drug recommendation with detailed analysis
    def show_recommendations(condition, patient_age, patient_gender, patient_medications, patient_conditions):
        result_html, instruction, df_display = get_drug_recommendations(condition, patient_age, patient_gender, patient_medications, patient_conditions)

        if df_display.empty:
            return result_html, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), condition
        else:
            return (
                result_html,
                gr.update(value=instruction, visible=True),
                gr.update(value=df_display, visible=True),
                gr.update(visible=True),
                condition
            )

    current_condition = gr.State("")

    recommend_btn.click(
        fn=show_recommendations,
        inputs=[condition_input, patient_age, patient_gender, patient_medications, patient_conditions],
        outputs=[recommendation_output, selection_instruction, drug_options_table, adverse_detail_display, current_condition]
    )

    # Drug selection (clicking on table)
    def handle_drug_selection(condition, age, gender, medications, evt: gr.SelectData):
        if evt.row_value and len(evt.row_value) > 0:
            drug_name = evt.row_value[0]  # First column is drug name

            # Get detailed analysis for the selected drug
            is_safe, detailed_interactions, interaction_summary = check_drug_safety(drug_name, medications)
            adverse_reactions = predict_adverse_reactions(drug_name, age, gender, medications)

            # Create detailed analysis display
            detail_html = f"""
            <div style="background: #1e293b; padding: 20px; border-radius: 15px; color: white; margin: 10px 0;">
                <h3 style="color: #60a5fa; margin: 0 0 15px 0;">🔍 Detailed Analysis: {drug_name}</h3>

                <div style="background: {'#14532d' if is_safe else '#7f1d1d'}; padding: 15px; border-radius: 10px; margin-bottom: 15px; border-left: 4px solid {'#22c55e' if is_safe else '#ef4444'};">
                    <h4 style="margin: 0 0 10px 0; color: white;">⚠️ Drug Interaction Analysis</h4>
                    <p style="margin: 0; font-size: 14px;">
                        <strong>Status:</strong> {'✅ SAFE - No interactions detected' if is_safe else '❌ INTERACTIONS DETECTED'}
                    </p>
                    {f'''
                    <div style="margin-top: 10px;">
                        <strong>Interacting Medications:</strong>
                        <ul style="margin: 5px 0; padding-left: 20px;">
                            {''.join([f'<li>{interaction["interacting_drug"]} - {interaction["description"]} ({interaction["severity"]})</li>' for interaction in detailed_interactions])}
                        </ul>
                    </div>
                    ''' if detailed_interactions else ''}
                </div>

                <div style="background: #374151; padding: 15px; border-radius: 10px; border-left: 4px solid #fbbf24;">
                    <h4 style="margin: 0 0 10px 0; color: #fbbf24;">🔬 Adverse Reaction Prediction</h4>
                    <p style="margin: 0; font-size: 14px; line-height: 1.6;">
                        {adverse_reactions}
                    </p>
                </div>

                <div style="background: #1f2937; padding: 15px; border-radius: 10px; margin-top: 15px; text-align: center;">
                    <p style="margin: 0; font-size: 14px; color: #94a3b8;">
                        <strong>Model Analysis:</strong> CSV lookup for interactions •
                        {'✅ AI Model for adverse reactions' if adverse_reaction_model else '📄 Clean "None" for unavailable predictions'}
                    </p>
                </div>
            </div>
            """

            # Update patient record
            success, message = update_patient_medication(selected_patient_id, drug_name, condition)

            if success:
                # Update history
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                patient_name = patients_df[patients_df['patient_id'] == selected_patient_id].iloc[0]['name']

                recommendations_history.append({
                    'timestamp': timestamp,
                    'patient_id': selected_patient_id,
                    'patient_name': patient_name,
                    'condition': condition,
                    'prescribed_drug': drug_name,
                    'interactions': detailed_interactions,
                    'adverse_reactions': adverse_reactions,
                    'safety_status': 'SAFE' if is_safe else 'WARNING'
                })

                # Update history display
                history_html = """
                <div style="background: #1e293b; padding: 20px; border-radius: 10px;">
                    <h3 style="color: #60a5fa; margin-top: 0;">📋 Recent Prescriptions</h3>
                """

                for rec in recommendations_history[-10:]:  # Show last 10
                    status_color = "#22c55e" if rec['safety_status'] == 'SAFE' else "#ef4444"
                    history_html += f"""
                    <div style="background: #334155; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 4px solid {status_color};">
                        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px; font-size: 14px; color: #e2e8f0;">
                            <div><strong>⏰ Time:</strong> {rec['timestamp']}</div>
                            <div><strong>👤 Patient:</strong> {rec['patient_name']}</div>
                            <div><strong>🏥 Condition:</strong> {rec['condition']}</div>
                            <div><strong>💊 Prescribed:</strong> <span style="color: {status_color}; font-weight: bold;">{rec['prescribed_drug']}</span></div>
                        </div>
                        <div style="margin-top: 10px; font-size: 12px; color: #94a3b8;">
                            <div><strong>Interactions:</strong> {', '.join([i['interacting_drug'] for i in rec.get('interactions', [])]) if rec.get('interactions') else 'None'}</div>
                            <div><strong>Adverse Reactions:</strong> {str(rec.get('adverse_reactions', 'None'))[:60]}...</div>
                        </div>
                    </div>
                    """

                history_html += "</div>"

                return (
                    f"""<div style="background: #22c55e; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                        <strong>✅ Prescription Successful!</strong> {message}
                    </div>""",
                    patients_df,
                    history_html,
                    detail_html
                )
            else:
                return (
                    f"""<div style="background: #ef4444; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                        <strong>❌ Error:</strong> {message}
                    </div>""",
                    patients_df,
                    gr.update(),
                    detail_html
                )

        return "No drug selected", patients_df, gr.update(), ""

    drug_options_table.select(
        fn=handle_drug_selection,
        inputs=[current_condition, patient_age, patient_gender, patient_medications],
        outputs=[status_message, dataset_display, history_display, adverse_detail_display]
    )



In [ ]:
# APPLICATION LAUNCH
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("🚀 Launching Professional Drug Recommendation System...")
    print("🔗 Loading with dark professional theme...")

    # Status check
    if patients_df.empty:
        print("❌ No patient data loaded - please check your ER_data.csv file")
    else:
        print(f"✅ Loaded {len(patients_df)} patients from ER database")

    if drug_model is None:
        print("❌ Drug prediction model not loaded - please check your model file")
    else:
        print("✅ Drug prediction model loaded successfully")

    if drug_interactions_available:
        print("✅ Drug interactions CSV is available and ready")
    else:
        print("❌ Drug interactions CSV not found - safety checking limited")

    # Adverse reaction model status
    if adverse_reaction_model:
        print("✅ Adverse reaction model loaded and active")
    else:
        print("📄 Adverse reaction model not available - will show 'None' for predictions")

    print("\n" + "="*60)
    print("🚀 SYSTEM READY FOR PRODUCTION USE!")
    print("📋 Features enabled:")
    print(f"   • Patient Database: {len(patients_df) if not patients_df.empty else 0} patients")
    print(f"   • Drug Predictions: {'✅' if drug_model else '❌'} (k=3)")
    if drug_interactions_available:
        print(f"   • Drug Interactions: ✅ (CSV database ready)")
    else:
        print(f"   • Drug Interactions: ❌ (CSV not found)")
    print(f"   • Adverse Reactions: {'✅ AI Model Active' if adverse_reaction_model else '📄 Clean None Response'}")
    print("="*60)

    # Launch with public sharing enabled
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        debug=True
    )


🚀 Launching Professional Drug Recommendation System...
🔗 Loading with dark professional theme...
✅ Loaded 1306 patients from ER database
✅ Drug prediction model loaded successfully
✅ Drug interactions CSV is available and ready
✅ Adverse reaction model loaded and active

🚀 SYSTEM READY FOR PRODUCTION USE!
📋 Features enabled:
   • Patient Database: 1306 patients
   • Drug Predictions: ✅ (k=3)
   • Drug Interactions: ✅ (CSV database ready)
   • Adverse Reactions: ✅ AI Model Active
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e5bf12639887ed6bb9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔍 Predicting drugs for condition: fever
📊 Using model type: <class '__main__.StreamlinedDrugSystem'>
🤖 Using predict_drugs_for_condition method
✅ Model returned 3 drugs: ['Propacetamol', 'Ibuprofen', 'Acetylsalicylic acid']

🔍 CHECKING DRUG INTERACTIONS
📋 Prescribed: Propacetamol
💊 Current meds: ['penicillin', 'metformin', 'acetaminophen', 'amoxicillin', 'galantamine', 'meperidine']
✅ CSV loaded: 4105 rows
✅ Found Propacetamol in database
🔗 Has 1783 potential interactions
⚠️ INTERACTION: Propacetamol ↔ penicillin
⚠️ INTERACTION: Propacetamol ↔ metformin
⚠️ INTERACTION: Propacetamol ↔ acetaminophen
⚠️ INTERACTION: Propacetamol ↔ amoxicillin
⚠️ INTERACTION: Propacetamol ↔ galantamine
⚠️ INTERACTION: Propacetamol ↔ meperidine
🤖 Generating adverse reactions for Propacetamol...
✅ Generated adverse reaction prediction

🔍 CHECKING DRUG INTERACTIONS
📋 Prescribed: Ibuprofen
💊 Current meds: ['penicillin', 'metformin', 'acetaminophen', 'amoxicillin', 'galantamine', 'meperidine']
✅ CSV loaded: 410